# Deploy the llama13B  model on Amazon SageMaker

As we have finetuned the model, next we will show you how to deploy the model on SageMaker.

In this notebook, we explore how to host a large language model on SageMaker using the [Large Model Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html) container that is optimized for hosting large models using DJLServing. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3") # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime") # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment() # jinja environment to generate model configuration templates

In [ ]:
# lookup the inference image uri based on our current region
djl_inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

**The model we tested is from https://huggingface.co/decapoda-research/llama-13b-hf/tree/main, you could download first and upload to S3, you could follow below commented code to download the model.**

In [ ]:
# !pip install huggingface_hub

In [ ]:
# from huggingface_hub import snapshot_download
# from pathlib import Path

# local_cache_path = Path("./model")
# local_cache_path.mkdir(exist_ok=True)

# model_name = "decapoda-research/llama-13b-hf"

# # Only download pytorch checkpoint files
# allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

# model_download_path = snapshot_download(
#     repo_id=model_name,
#     cache_dir=local_cache_path,
#     allow_patterns=allow_patterns,
# )

In [ ]:
# # Get the model files path
# import os
# from glob import glob

# local_model_path = None

# paths = os.walk(r'./model')
# for root, dirs, files in paths:
#     for file in files:
#         if file == 'config.json':
#             print(os.path.join(root,file))
#             local_model_path = str(os.path.join(root,file))[0:-11]
#             print(local_model_path)
# if local_model_path == None:
#     print("Model download may failed, please check prior step!")

In [ ]:
# %%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

# chmod +x ./s5cmd
# ./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/llama/pretrain/pinkmanlove/llama-7b-hf/ 

In [ ]:
# # remove model artifact from local notebook storage
# !rm -rf model

In [ ]:
pretrained_model_location = "s3://XXXXXXXXXXXX/XXXXXXXX/"# Change to the model artifact path in S3 which we get from the fine tune job
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

## Build the inference contianer image

In [ ]:
%%writefile Dockerfile.inference
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117 

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

## Install transfomers version which support LLaMaTokenizer
RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

In [ ]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-llama-inference-demo"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f Dockerfile.inference .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
## The image uri which is build and pushed above
inference_image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account_id, region, repo_name)
inference_image_uri

## Deploying a Large Language Model using Hugging Face Accelerate
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [ ]:
!mkdir accelerate_src

**IMPORTANT** The ```option.tensor_parallel_degree``` means how GPU we will use to load a single model, here we set it to 2, as 13B model need at least 26GB (fp16) GPU memory, here we use 2 GPU to host this 13B model.

In [ ]:
%%writefile accelerate_src/serving.template
engine=Python
option.entryPoint=djl_python.huggingface
option.s3url={{ s3url }}
option.task=text-generation
option.device_map=auto
option.dtype=fp16
option.tensor_parallel_degree=2

In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("accelerate_src/serving.template").open().read())
Path("accelerate_src/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
!pygmentize accelerate_src/serving.properties | cat -n

There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. 

We place the `serving.properties` file into a tarball and upload it to S3

In [ ]:
!tar czvf acc_model.tar.gz accelerate_src/ 

In [ ]:
s3_code_prefix = "llama/deploy/code/llama-13B"

code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [ ]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(
            image_uri=image_uri, 
              model_data=model_data, 
              role=role
             )
    
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=endpoint_name,
        model_data_download_timeout=60*15, ##
        container_startup_health_check_timeout=60*15 ##
        )
    
    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name, 
        sagemaker_session=sagemaker_session, 
        serializer=serializers.JSONSerializer(), 
        deserializer=deserializers.JSONDeserializer())
    
    return predictor

In [ ]:
# creates a unique endpoint name
endpoint_name = sagemaker.utils.name_from_base("llama-13B")
print(f"Our endpoint will be called {endpoint_name}")

In [ ]:
# deployment will take about 10 minutes
predictor = deploy_model(image_uri=inference_image_uri, 
                            model_data=code_artifact, 
                            role=role, 
                            endpoint_name=endpoint_name, 
                            instance_type="ml.g5.12xlarge", 
                            sagemaker_session=sess)

Let's run an example with a basic text generation prompt Large model inference is

In [ ]:
%%time

print(predictor.predict({ 
                    "inputs" : "Large model inference is", 
#                     "parameters": { "max_length": 128, "temperature": 0.5 }
                        "parameters": { "max_new_tokens": 128, "min_new_tokens": 128 }

                })[0]["generated_text"])

In [ ]:
print(predictor.predict({ "inputs":
                                """Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment:""",
                      "parameters": { 
                          "max_length": 50, 
                          "temperature": 0.5,
#                           "max_new_tokens": 128, 
#                           "min_new_tokens": 128 
                      }
                     }
                    )[0]["generated_text"])

Finally Let's do a quick benchmark to see what kind of latency we can expect from this model

In [ ]:
%%timeit -n3 -r1
predictor.predict({ 
                    "inputs" : "Large model inference is", 
                     "parameters": { 
                          "max_length": 50, 
                          "temperature": 0.5,
#                           "max_new_tokens": 128, 
#                           "min_new_tokens": 128 
                      }
                })

In [ ]:
# Clean up the endpoint before proceeding
predictor.delete_endpoint()

## Reference

[sagemaker-hosting/Large-Language-Model-Hosting/](https://github.com/aws-samples/sagemaker-hosting/tree/main/Large-Language-Model-Hosting)